# Data Preprocessing (Cleaning, Revisualize)

##### Loading the Libraries and Dataset

In [97]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick

# Load dataset
df_clean = pd.read_csv("../../data/tech_salary_data.csv")

##### Drop Irrelevant Columns and Level Column

In [ ]:
# Drop Irrelevant Columns
cols_to_drop = ["basesalary", "stockgrantvalue", "bonus", "timestamp", "tag", "otherdetails",
                 "cityid", "dmaid", "rowNumber"]
for col in cols_to_drop:
    if col in df_clean.columns:
        df_clean.drop(columns=col, inplace=True)

# Drop 'level' column (too complex and not useful)
df_clean.drop(columns=['level'], inplace=True)

##### Removing Rows where Race is NaN/Null/Empty/Unknown

In [99]:
# Keep only rows with known Race
df_clean.dropna(subset=["Race"], inplace=True)

##### Removing Rows where Gender is not Male/Female

In [100]:
# Keep only rows with Male/Female Gender
df_clean["gender"] = df_clean["gender"].str.strip().str.title()
df_clean = df_clean[df_clean["gender"].isin(["Male", "Female"])]

##### Removing Rows where Company is Null

In [101]:
df_clean = df_clean[df_clean["company"].notna()]

# Also fixing multiple levels of Google in company column
import numpy as np

df_clean["company"] = np.where(
df_clean["company"].astype("string").str.contains("google", case=False, na=False),
"Google",
df_clean["company"]
)

##### Changing Null Values in Education to Other

In [102]:
# Fill missing Education with "Other" value
if "Education" in df_clean.columns:
    df_clean["Education"] = df_clean["Education"].fillna("Other")

##### Changing Specific Location to Country

In [ ]:
df_clean['country'] = df_clean['location'].apply(lambda x: str(x).split(',')[-1].strip())

# List of US state abbreviations
us_states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY',
             'LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND',
             'OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','DC']

# Replace state codes with "United States"
df_clean['country'] = df_clean['country'].apply(lambda x: 'United States' if x in us_states else x)

country_map = {
    "UK": "United Kingdom",
    "U.K.": "United Kingdom",
    "England": "United Kingdom",
    "Scotland": "United Kingdom",
    "CA": "United States",  # handled above but just in case
    "Hong Kong": "China",
    "Russia": "Russian Federation"
    # add more as you see in your dataset
}

df_clean['country'] = df_clean['country'].replace(country_map)

# Dropping the original 'location' column as we have extracted 'country'
df_clean.drop(columns=['location'], inplace=True)

##### Synchronizing Education Binary Variables
Masters_Degree, Bachelors_Degree, etc.

In [104]:
# Synchronize education binary variables
edu_bins = [
    ("Doctorate",       "Doctorate_Degree"),
    ("Masters",         "Masters_Degree"),
    ("Bachelors",       "Bachelors_Degree"),
    ("Some_College",    "Some_College"),
    ("Highschool",      "Highschool")
]

def fix_education(row):
    flags = row[[col for _, col in edu_bins]]
    if flags.sum() == 1:
        return row
    row[[col for _, col in edu_bins]] = 0
    level = str(row.get("Education", "")).strip().title()
    for label, col in edu_bins:
        if level.startswith(label):
            row[col] = 1
            break
    return row

df_clean = df_clean.apply(fix_education, axis=1)

# Dropping individual binary education columns
df_clean.drop(columns=["Masters_Degree", 
                       "Bachelors_Degree", 
                       "Doctorate_Degree",
                       "Highschool",
                       "Some_College"], inplace=True)

##### Duplication of Dataframe for Outlier Elimination (Tukey's Fence Method)

In [105]:
df_cleaned_v2 = df_clean.copy()

##### Identifying Numeric Columns and Computing Q1, Q3 and IQR for each Column

In [106]:
numeric_cols = [
    'totalyearlycompensation',
    'yearsofexperience',
    'yearsatcompany'
]

Q1 = df_cleaned_v2[numeric_cols].quantile(0.25)
Q3 = df_cleaned_v2[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

##### Creating Mask for Outliers

In [107]:
# Build Mask - True if row is within [Q1 - 1.5*IQR, Q3 + 1.5IQR] for EVERY numeric column
fence_low  = Q1 - 1.5 * IQR
fence_high = Q3 + 1.5 * IQR

outlier_mask = pd.concat([
    (df_cleaned_v2[col] < fence_low[col]) |
    (df_cleaned_v2[col] > fence_high[col])
    for col in numeric_cols
], axis=1).any(axis=1)

# Negating it to get non-outliers (using logical not)
mask = ~outlier_mask

n_before = len(df_cleaned_v2)
n_after = mask.sum()
print(f"Dropping {n_before-n_after} / {n_before} rows as outliers ({(n_before-n_after)/n_before:.1%})")

Dropping 1957 / 22198 rows as outliers (8.8%)


##### Copying Outlier-free Data to new Data Frame

In [108]:
df_no_outliers = df_cleaned_v2.loc[mask].copy()

##### Removing Duplicate Entries

In [109]:
df_no_outliers = df_no_outliers.drop_duplicates()

##### Comparison of Original and Preprocessed Data and Exporting the Preprocessed Data

In [110]:
# Show the final shape of clean dataset and export to csv
print("Before cleaning: 62,642 rows × 24 columns.")
print(f"After cleaning: {df_no_outliers.shape[0]:,} rows × {df_no_outliers.shape[1]} columns.")

# Export data to new csv file
df_no_outliers.to_csv("../../data/tech_salary_data_CLEANED.csv", index=False)
print("Dataset saved to tech_salary_data_CLEANED.csv.")

Before cleaning: 62,642 rows × 24 columns.
After cleaning: 19,878 rows × 9 columns.
Dataset saved to tech_salary_data_CLEANED.csv.


##### Column Types and Missing Values of Preprocessed Data

In [ ]:
# Importing cleaned data
df = pd.read_csv("../../data/tech_salary_data_CLEANED.csv")

# Show the column types and non-null counts
print("=== Columns and Data Types ===")
print(df.info(), "\n")

# Show the missing value counts
print("=== Missing Value Counts ===")
print(df.isnull().sum(), "\n")

=== Columns and Data Types ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19878 entries, 0 to 19877
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  19878 non-null  object 
 1   title                    19878 non-null  object 
 2   totalyearlycompensation  19878 non-null  int64  
 3   yearsofexperience        19878 non-null  float64
 4   yearsatcompany           19878 non-null  float64
 5   gender                   19878 non-null  object 
 6   Race                     19878 non-null  object 
 7   Education                19878 non-null  object 
 8   country                  19878 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.4+ MB
None 

=== Missing Value Counts ===
company                    0
title                      0
totalyearlycompensation    0
yearsofexperience          0
yearsatcompany             0
gender                  